In [ ]:
attr = ["Tank1WaterLevel", "Tank2WaterLevel", "Tank2OutFlow", "Tank3OutFlow", "Tank2.level", "wt3_valve"]
attr = ["Tank1WaterLevel", "Tank2WaterLevel", "Tank2OutFlow", "Tank3OutFlow", "Tank2.level", "wt3_valve"]
attr = ["Tank3OutFlow", "Tank2.level", "wt3_valve", "Tank1WaterLevel", "Tank2WaterLevel", "Tank2OutFlow",  ]
attr = ["Tank3OutFlow", "Tank2.level", "wt3_valve", "Tank1WaterLevel", "Tank2WaterLevel", "Tank2OutFlow",  ]
attr = ["Tank1WaterLevel", "Tank2WaterLevel", "Tank1.Tank1InFlow", "Tank1.Tank2OutFlow", "wt3_valve", "Tank2.level"]
